<a href="https://colab.research.google.com/github/MELAI-1/Quantum-computing-project/blob/main/QNN_CLAMP_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Download the dataset**

In [2]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 43.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

# **Load the data**

In [4]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

# **Data visualization**

In [6]:
df.head()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,256,4,1,0,1,...,1,1,3,0,6.603616,5.443362,1181520,6.627552,1,0
1,144,3,4,65535,184,184,4,1,0,1,...,1,1,3,0,5.205926,2.123522,7680,5.318221,0,0
2,144,3,4,65535,184,272,5,1,0,1,...,1,1,4,0,6.238000,3.380859,57872,6.507758,1,0
3,144,3,4,65535,184,184,1,1,0,1,...,1,0,1,0,0.000000,0.000000,95616,4.575092,1,0
4,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.355626,0.702621,48128,5.545531,1,0


In [9]:
df.columns

Index(['e_cblp', 'e_cp', 'e_cparhdr', 'e_maxalloc', 'e_sp', 'e_lfanew',
       'NumberOfSections', 'CreationYear', 'FH_char0', 'FH_char1', 'FH_char2',
       'FH_char3', 'FH_char4', 'FH_char5', 'FH_char6', 'FH_char7', 'FH_char8',
       'FH_char9', 'FH_char10', 'FH_char11', 'FH_char12', 'FH_char13',
       'FH_char14', 'MajorLinkerVersion', 'MinorLinkerVersion', 'SizeOfCode',
       'SizeOfInitializedData', 'SizeOfUninitializedData',
       'AddressOfEntryPoint', 'BaseOfCode', 'BaseOfData', 'ImageBase',
       'SectionAlignment', 'FileAlignment', 'MajorOperatingSystemVersion',
       'MinorOperatingSystemVersion', 'MajorImageVersion', 'MinorImageVersion',
       'MajorSubsystemVersion', 'MinorSubsystemVersion', 'SizeOfImage',
       'SizeOfHeaders', 'CheckSum', 'Subsystem', 'OH_DLLchar0', 'OH_DLLchar1',
       'OH_DLLchar2', 'OH_DLLchar3', 'OH_DLLchar4', 'OH_DLLchar5',
       'OH_DLLchar6', 'OH_DLLchar7', 'OH_DLLchar8', 'OH_DLLchar9',
       'OH_DLLchar10', 'SizeOfStackReserve', 'SizeO

# **Data preprocessing**

first we will remove the packer_type variable

In [11]:
# df.drop(['packer_type'],axis=1,inplace=True)

In [12]:
from sklearn.utils import shuffle
df = shuffle(df)

In [13]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [14]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [15]:
df.drop(['index'],axis=1,inplace=True)

In [16]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,224,4,1,0,1,...,1,1,3,0,5.941287,3.465646,47104,5.835979,1,1
1,144,8,4,65535,40,128,5,1,1,1,...,1,1,4,0,5.960037,5.062149,61440,6.123409,0,1
2,144,3,4,65535,184,216,4,1,1,1,...,1,0,4,1,6.661104,5.002675,163328,7.682310,1,1
3,144,3,4,65535,184,248,5,1,1,1,...,1,1,4,0,3.528462,0.000000,123904,5.184974,0,1
4,128,1,4,65535,320,128,5,1,1,1,...,1,1,4,0,6.200870,4.150104,18944,5.435619,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,208,3,1,0,1,...,1,2,1,1,0.000000,0.000000,90112,7.946483,0,1
5206,144,3,4,65535,184,232,4,1,1,1,...,1,0,4,0,6.446074,1.812934,118784,6.004108,1,0
5207,144,3,4,65535,184,184,1,1,0,1,...,1,0,1,0,0.000000,0.000000,2560,3.792644,1,0
5208,144,3,4,65535,184,256,4,1,0,1,...,1,1,3,0,6.760367,2.035346,268552,6.742753,1,0


In [17]:
df_5percent=df[0:5210]

In [18]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,224,4,1,0,1,...,1,1,3,0,5.941287,3.465646,47104,5.835979,1,1
1,144,8,4,65535,40,128,5,1,1,1,...,1,1,4,0,5.960037,5.062149,61440,6.123409,0,1
2,144,3,4,65535,184,216,4,1,1,1,...,1,0,4,1,6.661104,5.002675,163328,7.682310,1,1
3,144,3,4,65535,184,248,5,1,1,1,...,1,1,4,0,3.528462,0.000000,123904,5.184974,0,1
4,128,1,4,65535,320,128,5,1,1,1,...,1,1,4,0,6.200870,4.150104,18944,5.435619,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,208,3,1,0,1,...,1,2,1,1,0.000000,0.000000,90112,7.946483,0,1
5206,144,3,4,65535,184,232,4,1,1,1,...,1,0,4,0,6.446074,1.812934,118784,6.004108,1,0
5207,144,3,4,65535,184,184,1,1,0,1,...,1,0,1,0,0.000000,0.000000,2560,3.792644,1,0
5208,144,3,4,65535,184,256,4,1,0,1,...,1,1,3,0,6.760367,2.035346,268552,6.742753,1,0


In [19]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [20]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [23]:
df_5percent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5210 entries, 0 to 5209
Data columns (total 69 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   e_cblp                       5210 non-null   float64
 1   e_cp                         5210 non-null   float64
 2   e_cparhdr                    5210 non-null   float64
 3   e_maxalloc                   5210 non-null   float64
 4   e_sp                         5210 non-null   float64
 5   e_lfanew                     5210 non-null   float64
 6   NumberOfSections             5210 non-null   float64
 7   CreationYear                 5210 non-null   float64
 8   FH_char0                     5210 non-null   float64
 9   FH_char1                     5210 non-null   float64
 10  FH_char2                     5210 non-null   float64
 11  FH_char3                     5210 non-null   float64
 12  FH_char4                     5210 non-null   float64
 13  FH_char5          

In [22]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.0,...,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5.210000e+03,5210.000000,5210.000000,5210.000000
mean,152.658733,10.633589,8.903263,65137.618234,202.530902,222.833013,4.662956,0.990403,0.362764,1.0,...,0.999040,1.360845,3.302111,0.156430,4.939586,2.523549,7.875718e+05,6.364756,0.544146,0.522457
std,616.499070,392.745064,251.685103,5005.606042,968.625098,48.591497,1.922882,0.097502,0.480844,0.0,...,0.030967,1.623972,1.146956,0.363297,2.521787,2.685909,5.293440e+06,1.119881,0.498095,0.499543
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.886006,0.000000,6.144000e+04,5.695785,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.148211,1.555973,1.218560e+05,6.390618,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.507072,4.685836,3.055480e+05,7.314531,1.000000,1.000000
max,37008.000000,20050.000000,12851.000000,65535.000000,65534.000000,648.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,8.000000,1.000000,7.999859,7.999620,1.657081e+08,7.999997,1.000000,1.000000


In [24]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [25]:
# Reduce dimensions using PCA to fit the dimensions with the qubits
n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [26]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [27]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

,proportion
class,
1.0,55.625
0.0,44.375


In [28]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

,proportion
class,
1.0,57.5
0.0,42.5


In [29]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)

    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')

    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))

    return qml.expval(qml.PauliZ(0))

In [30]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00543743 -0.0061543  -0.01197806]
  [ 0.02141658 -0.00094621 -0.00928828]]

 [[-0.0088523  -0.00412188 -0.00482619]
  [ 0.00164165  0.00233095  0.00117995]]

 [[ 0.01462378  0.01538715 -0.02439106]
  [ 0.00603441 -0.00251044 -0.00163867]]

 [[-0.0147633   0.01486981 -0.00024455]
  [ 0.00355551  0.00417011  0.00832462]]

 [[-0.00293399 -0.00029839  0.00095126]
  [ 0.00664654 -0.00140218 -0.00033193]]] 0.0


In [31]:
circuit(weights_init, data[0])

tensor(-0.00693554, requires_grad=True)

In [32]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [33]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [34]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [35]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [36]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.08290204, requires_grad=False), tensor(-0.12922103, requires_grad=False)], Y =  1
X = [tensor(-0.08290193, requires_grad=False), tensor(-0.11963957, requires_grad=False)], Y =  1
X = [tensor(-0.08181418, requires_grad=False), tensor(-0.09275436, requires_grad=False)], Y =  1
X = [tensor(-0.08290153, requires_grad=False), tensor(-0.06715157, requires_grad=False)], Y =  1
X = [tensor(-0.08270061, requires_grad=False), tensor(-0.12970498, requires_grad=False)], Y =  1


In [37]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [40]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 0.9844036 | Accuracy: 0.6125000 
Iter:     2 | Cost: 0.9746896 | Accuracy: 0.6125000 
Iter:     3 | Cost: 0.9767145 | Accuracy: 0.5562500 
Iter:     4 | Cost: 0.9657871 | Accuracy: 0.5562500 
New best
Iter:     5 | Cost: 0.9566465 | Accuracy: 0.6250000 
New best
Iter:     6 | Cost: 0.9500437 | Accuracy: 0.6312500 
New best
Iter:     7 | Cost: 0.9441556 | Accuracy: 0.6375000 
Iter:     8 | Cost: 0.9347946 | Accuracy: 0.6250000 
Iter:     9 | Cost: 0.9542435 | Accuracy: 0.5812500 
Iter:    10 | Cost: 0.9953476 | Accuracy: 0.5750000 
Iter:    11 | Cost: 0.9801586 | Accuracy: 0.5750000 
Iter:    12 | Cost: 0.9407598 | Accuracy: 0.6062500 
Iter:    13 | Cost: 0.9185248 | Accuracy: 0.6250000 
Iter:    14 | Cost: 0.9175324 | Accuracy: 0.6375000 
New best
Iter:    15 | Cost: 0.9279527 | Accuracy: 0.6687500 
Iter:    16 | Cost: 0.9455405 | Accuracy: 0.6062500 
Iter:    17 | Cost: 0.9865021 | Accuracy: 0.4687500 
Iter:    18 | Cost: 0.9552030 | Accuracy: 0.5125000 
I

In [41]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [42]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.4937706813828107, Accuracy: 45.0%


In [43]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,-1.0,1.0
1,1.0,-1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,-1.0
5,1.0,-1.0
6,1.0,-1.0
7,1.0,1.0
8,1.0,-1.0
9,1.0,1.0


In [44]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         5
        -1.0       0.00      0.00      0.00         5
         1.0       0.78      0.51      0.62        35
         1.0       0.78      0.51      0.62        35

    accuracy                           0.45        80
   macro avg       0.39      0.26      0.31        80
weighted avg       0.68      0.45      0.54        80

0.782608695652174
0.5142857142857142
0.6206896551724138
0.2571428571428571


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
